## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page: [MeSH Clinical Terminology Mapper](https://aws.amazon.com/marketplace/pp/prodview-rfrl67uzxbhzk)
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## MeSH Clinical Terminology Mapper

- **Model**: `mesh_vdb_resolver`
- **Model Description**: This pretrained pipeline extracts clinical entities from clinical text and maps them to their corresponding Medical Subject Heading (MeSH) codes.

In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [ ]:
import json
import os
import boto3
import pandas as pd
import sagemaker as sage
from sagemaker import ModelPackage
from sagemaker import get_execution_role
from IPython.display import display
from urllib.parse import urlparse


In [3]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [4]:
model_name = "mesh-vdb-resolver"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.2xlarge"

## 2. Create a deployable model from the model package.

In [5]:
model = ModelPackage(
    role=role, 
    model_package_arn=model_package_arn,
    sagemaker_session=sagemaker_session,
)

### Input Format

To use the model, you need to provide input in one of the following supported formats:

#### JSON Format

Provide input as JSON. We support two variations within this format:

1. **Array of Text Documents**: 
   Use an array containing multiple text documents. Each element represents a separate text document.

   ```json
   {
       "text": [
           "Text document 1",
           "Text document 2",
           ...
       ]
   }

    ```

2. **Single Text Document**:
   Provide a single text document as a string.


   ```json
    {
        "text": "Single text document"
    }
   ```

#### JSON Lines (JSONL) Format

Provide input in JSON Lines format, where each line is a JSON object representing a text document.

```
{"text": "Text document 1"}
{"text": "Text document 2"}
```

## 3. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

### A. Deploy the SageMaker model to an endpoint

In [6]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type, 
    endpoint_name=model_name,
)

--------------!

Once endpoint has been created, you would be able to perform real-time inference.

In [7]:
def invoke_realtime_endpoint(record, content_type="application/json", accept="application/json"):
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=json.dumps(record) if content_type == "application/json" else record,
    )

    response_body = response["Body"].read().decode("utf-8")

    if accept == "application/json":
        return json.loads(response_body)
    elif accept == "application/jsonlines":
        return response_body
    else:
        raise ValueError(f"Unsupported accept type: {accept}")

### Initial Setup

In [8]:
docs = [
    """The patient is a 40-year-old white male who presents with a chief complaint of "chest pain". The patient is diabetic and has a prior history of coronary artery disease. The patient presents today stating that his chest pain started yesterday evening. He has been advised Aspirin 81 milligrams QDay. Humulin N. insulin 50 units in a.m. Hydrochlorothiazide 50 mg QDay. Nitroglycerin 1/150 sublingually PRN chest pain.""",
    """ A 5-month-old boy brought by his parents because of 2 days of cough. Mother took him when cough started 2 days go to Clinic. But cough got worse and he also started having fever yesterday at night. Mother did not measure it. REVIEW OF SYSTEMS: No vomiting. No diarrhea. No skin rash. No cyanosis. """,
]

sample_text = """She was admitted to the hospital with chest pain and found to have bilateral pleural effusion, the right greater than the left. We reviewed the pathology obtained from the pericardectomy in March 2006, which was diagnostic of mesothelioma. At this time, chest tube placement for drainage of the fluid occurred and thoracoscopy with fluid biopsies, which were performed, which revealed malignant mesothelioma."""

### JSON

In [9]:
input_json_data = {"text": sample_text}
response_json = invoke_realtime_endpoint(input_json_data, content_type="application/json", accept="application/json")
pd.DataFrame(response_json["predictions"][0])

,begin,end,ner_chunk,ner_label,ner_confidence,concept_code,resolution,score,all_codes,all_resolutions,all_score
0,38,47,chest pain,PROBLEM,0.9943,D002637,chest pain,1.000000,"[D002637, D002637, D002637, D002637, D013898]","[chest pain, chest pains, pain, chest, pains, chest, chest injury]","[1.0000003576278687, 0.922394871711731, 0.9001147150993347, 0.8813276290893555, 0.7900357246398926]"
1,67,92,bilateral pleural effusion,PROBLEM,0.9502334,D010996,pleural effusion,0.825787,"[D010996, D010996, D010996, D010996, D016066]","[pleural effusion, pleural effusions, effusion, pleural, effusions, pleural, effusion, malignant pleural]","[0.8257865905761719, 0.8088810443878174, 0.796427845954895, 0.7801889777183533, 0.7638202905654907]"
2,140,152,the pathology,TEST,0.44089997,D010336,pathology,0.919547,"[D010336, D010340, D010338, D010341, D057089]","[pathology, pathology, surgical, pathology, clinical, pathology, veterinary, pathology, molecular]","[0.9195465445518494, 0.8671680092811584, 0.8643243312835693, 0.8459447026252747, 0.8372940421104431]"
3,168,185,the pericardectomy,TREATMENT,0.9546,D010492,pericardectomy,0.955846,"[D010492, D010492, D010492, D010492, D010492]","[pericardectomy, pericardectomies, pericardotomy, pericardotomies, pericardiectomy]","[0.9558460116386414, 0.9290467500686646, 0.8579220175743103, 0.8520034551620483, 0.8384276628494263]"
4,226,237,mesothelioma,PROBLEM,0.961,D000086002,malignant mesothelioma,0.904331,"[D000086002, D000086002, D000086002, D054363, D000086002]","[malignant mesothelioma, malignant mesotheliomas, malignant pleural mesothelioma, fibrous mesothelioma, mesothelioma, malignant]","[0.9043309688568115, 0.8989654779434204, 0.8591769933700562, 0.8589586019515991, 0.8569139242172241]"
5,254,273,chest tube placement,TREATMENT,0.95446664,D015505,chest tubes,0.870246,"[D015505, D056446, D056446, D056446, D000069258]","[chest tubes, knee-chest position, knee chest position, knee-chest positions, chest aspirations]","[0.870246410369873, 0.7561070919036865, 0.7526574730873108, 0.7446746826171875, 0.740543007850647]"
6,279,299,drainage of the fluid,PROBLEM,0.746675,D065634,"fluid drainage, cerebrospinal",0.858006,"[D065634, D065634, D004322, D065634, D065634]","[fluid drainage, cerebrospinal, drainage, cerebrospinal fluid, drainage, fluid drainages, cerebrospinal, cerebrospinal fluid drainage]","[0.8580055236816406, 0.8529242873191833, 0.8477861881256104, 0.8446167707443237, 0.838034987449646]"
7,314,325,thoracoscopy,TREATMENT,0.7614,D013906,thoracoscopy,1.000000,"[D013906, D013906, D020708, D013906, D013906]","[thoracoscopy, thoracoscopies, thoracoscopes, thoracoscopic surgery, thoracoscopic surgical procedures]","[1.0, 0.9153507351875305, 0.8540973663330078, 0.8537589907646179, 0.8490511178970337]"
8,332,345,fluid biopsies,TEST,0.7463,D001706,biopsies,0.891502,"[D001706, D001707, D000073890, D001707, D044963]","[biopsies, biopsies, puncture, liquid biopsy, puncture biopsies, fine needle biopsies]","[0.891502320766449, 0.8405726552009583, 0.8362082839012146, 0.8332265615463257, 0.8241764903068542]"
9,385,406,malignant mesothelioma,PROBLEM,0.9796,D000086002,malignant mesothelioma,1.000001,"[D000086002, D000086002, D000086002, D000086002, D000086002]","[malignant mesothelioma, malignant mesotheliomas, malignant pleural mesothelioma, malignant pleural mesotheliomas, mesothelioma, malignant]","[1.0000007152557373, 0.9908357858657837, 0.9494664669036865, 0.9431859254837036, 0.927587628364563]"


### JSON Lines

In [11]:
def create_jsonl(records):
    if isinstance(records, str):
        records = [records]
    json_records = [{"text": text} for text in records]
    json_lines = "\n".join(json.dumps(record) for record in json_records)
    return json_lines

In [12]:
input_jsonl_data = create_jsonl(sample_text)
data = invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"predictions": [{"begin": 38, "end": 47, "ner_chunk": "chest pain", "ner_label": "PROBLEM", "ner_confidence": "0.9943", "concept_code": "D002637", "resolution": "chest pain", "score": 1.0000003576278687, "all_codes": ["D002637", "D002637", "D002637", "D002637", "D013898"], "all_resolutions": ["chest pain", "chest pains", "pain, chest", "pains, chest", "chest injury"], "all_score": [1.0000003576278687, 0.922394871711731, 0.9001147150993347, 0.8813276290893555, 0.7900357246398926]}, {"begin": 67, "end": 92, "ner_chunk": "bilateral pleural effusion", "ner_label": "PROBLEM", "ner_confidence": "0.9502334", "concept_code": "D010996", "resolution": "pleural effusion", "score": 0.8257865905761719, "all_codes": ["D010996", "D010996", "D010996", "D010996", "D016066"], "all_resolutions": ["pleural effusion", "pleural effusions", "effusion, pleural", "effusions, pleural", "effusion, malignant pleural"], "all_score": [0.8257865905761719, 0.8088810443878174, 0.796427845954895, 0.7801889777183533, 0

### B. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [14]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Batch inference

In [15]:
validation_json_file_name = "input.json"
validation_jsonl_file_name = "input.jsonl"

validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/jsonl/"

def upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/{file_format[1:]}/{file_name}",
        Body=input_data.encode("UTF-8"),
    )

In [16]:
# Create JSON and JSON Lines data
input_jsonl_data = create_jsonl(docs)
input_json_data = json.dumps({"text": docs})

# Upload JSON and JSON Lines data to S3
upload_to_s3(input_json_data, validation_json_file_name)
upload_to_s3(input_jsonl_data, validation_jsonl_file_name)

### JSON

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [ ]:
def retrieve_json_output_from_s3(validation_file_name):
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    display(data)

In [21]:
retrieve_json_output_from_s3(validation_json_file_name)

{'predictions': [[{'begin': 80,
    'end': 89,
    'ner_chunk': 'chest pain',
    'ner_label': 'PROBLEM',
    'ner_confidence': '0.81095004',
    'concept_code': 'D002637',
    'resolution': 'chest pain',
    'score': 1.000000238418579,
    'all_codes': ['D002637', 'D002637', 'D002637', 'D002637', 'D013898'],
    'all_resolutions': ['chest pain',
     'chest pains',
     'pain, chest',
     'pains, chest',
     'chest injury'],
    'all_score': [1.000000238418579,
     0.9223949313163757,
     0.9001147747039795,
     0.8813276290893555,
     0.7900357246398926]},
   {'begin': 108,
    'end': 115,
    'ner_chunk': 'diabetic',
    'ner_label': 'PROBLEM',
    'ner_confidence': '0.9946',
    'concept_code': 'D002271',
    'resolution': 'diabetal',
    'score': 0.9493172764778137,
    'all_codes': ['D002271', 'D005905', 'C070071', 'D017719', 'D017719'],
    'all_resolutions': ['diabetal',
     'diabeta',
     'diabeton',
     'foot, diabetic',
     'diabetic foot'],
    'all_score': [0.949

### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [ ]:
def retrieve_jsonlines_output_from_s3(validation_file_name):

    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

In [24]:
retrieve_jsonlines_output_from_s3(validation_jsonl_file_name)

{"predictions": [{"begin": 80, "end": 89, "ner_chunk": "chest pain", "ner_label": "PROBLEM", "ner_confidence": "0.81095004", "concept_code": "D002637", "resolution": "chest pain", "score": 1.000000238418579, "all_codes": ["D002637", "D002637", "D002637", "D002637", "D013898"], "all_resolutions": ["chest pain", "chest pains", "pain, chest", "pains, chest", "chest injury"], "all_score": [1.000000238418579, 0.9223949313163757, 0.9001147747039795, 0.8813276290893555, 0.7900357246398926]}, {"begin": 108, "end": 115, "ner_chunk": "diabetic", "ner_label": "PROBLEM", "ner_confidence": "0.9946", "concept_code": "D002271", "resolution": "diabetal", "score": 0.9493172764778137, "all_codes": ["D002271", "D005905", "C070071", "D017719", "D017719"], "all_resolutions": ["diabetal", "diabeta", "diabeton", "foot, diabetic", "diabetic foot"], "all_score": [0.9493172764778137, 0.8931193351745605, 0.8634507656097412, 0.8435543775558472, 0.8424345850944519]}, {"begin": 144, "end": 166, "ner_chunk": "corona

In [25]:
model.delete_model()

INFO:sagemaker:Deleting model with name: mesh-vdb-resolver-2-2025-08-18-12-11-17-522


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

